In [1]:
# Step 1: Install required libraries
!pip install -q langchain langchain-community langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.0 MB/s eta 0:00:00


In [43]:
# Step 2: Import necessary modules
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
    PromptTemplate,
    AIMessagePromptTemplate
)
from langchain_core.output_parsers import StrOutputParser
import os
import getpass

In [3]:
# Step 3: Set up your Google API key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [4]:
# Step 4: Initialize the Gemini model
chat_model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")  # Use 'gemini-pro' as the model

In [44]:
example_prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("{question}"),
    AIMessagePromptTemplate.from_template("{answer}")
])

In [45]:
examples = [
    {
        "question": "How to get from Isfahan to Tehran?",
        "answer": """
        option 1: mode=bus, min_time_in_min=360 max_time_in_min=390 description=Take bus from Kaveh Terminal.
        option 2: mode=plane min_time_in_min=60 max_time_in_min=70 description=Take plane from Iran Air, Mahan Air or Zagros Air.
        option 3: mode=taxi min_time_in_min=330 max_time_in_min=360 description=Take private taxi using Snapp.
        """
    },
    {
        "question": "How to get from Shiraz to Isfahan?",
        "answer": """
        option 1: mode=bus, min_time_in_min=360 max_time_in_min=390 description=Take bus from Karandish Terminal.
        option 2: mode=plane min_time_in_min=75 max_time_in_min=90 description=Take plane from Iran Air.
        option 3: mode=taxi min_time_in_min=320 max_time_in_min=350 description=Take private taxi using Snapp.
        """
    },
]

In [46]:
fewshot_prompt_template = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    input_variables=["question"]
)

In [47]:
system_prompt_template = PromptTemplate(
    template = "You are a travel expert helping to find efficient and comfortable way of travelling from point A to point B."
)

system_message_template = SystemMessagePromptTemplate(prompt=system_prompt_template)

In [48]:
user_prompt_template = PromptTemplate(
    template = "how to reach {target} from {source}?",
    input_variables=["target", "source"]
)

user_message_template = HumanMessagePromptTemplate(prompt=user_prompt_template)

In [49]:
chat_prompt_template_with_fewshot = ChatPromptTemplate.from_messages([
    system_message_template,
    fewshot_prompt_template,
    user_message_template
])

In [50]:
user_input_source = "Tehran" # @param{type: "string"}

In [51]:
user_input_target = "Yazd" # @param{type: "string"}

In [52]:
messages = chat_prompt_template_with_fewshot.format_messages(target=user_input_target, source=user_input_source)

In [53]:
chat_model_response= chat_model.invoke(messages)

In [54]:
parser = StrOutputParser()
response = parser.invoke(chat_model_response)

In [55]:
print(response)

You have several options to travel from Tehran to Yazd:

* **Train:**  This is a popular and generally comfortable option.  There are various classes available, including comfortable overnight sleeper trains. The journey takes about 6-8 hours.  You can book tickets online or at the train station.

* **Bus:**  Buses are a more affordable option than trains, with frequent departures throughout the day and night.  VIP buses offer more comfortable seating and amenities. The bus journey takes around 6-7 hours.  Tickets can be purchased at bus terminals or online.

* **Plane:**  Flying is the fastest way to get to Yazd, with a flight time of approximately 1 hour.  Yazd Airport (AZD) is located a short distance from the city center.  Consider the added time for airport transfers and check-in procedures.

* **Private Car/Taxi:** Hiring a private car or taxi is the most expensive option but offers flexibility and door-to-door service. The drive takes about 6-7 hours.  You can arrange this throu